<a href="https://colab.research.google.com/github/hydrus3109/PRIMES_AIprivacy/blob/main/Pair_wise_stability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries

In [ ]:
!pip install opacus
!pip install e2cnn
from e2cnn import gspaces
from e2cnn import nn
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets
import torchvision.transforms as transforms
import torch.optim as optim
!pip install opacus
from torch.utils.data import Dataset
from torchvision.transforms import RandomRotation
from torchvision.transforms import Pad
from torchvision.transforms import Resize
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


model



In [ ]:
from torch.cuda.random import device_count
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.data import random_split
from torch.utils.data import Dataset
from torchvision.transforms import RandomRotation
from torchvision.transforms import Pad
from torchvision.transforms import Resize
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset =CIFAR10(root='./', train=True,download=True,transform = transform)
train = list(range(10, 1000,1))
valid = list(range(1000, 2000,1))
test = list(range(0,1000,1))

trainset_1 = torch.utils.data.Subset(trainset, train)
trainloader_1 = torch.utils.data.DataLoader(trainset_1, shuffle=True, num_workers=4,batch_size = 20)
validset = torch.utils.data.Subset(trainset, valid)
validloader = torch.utils.data.DataLoader(validset, shuffle=True, num_workers=4,batch_size = 20)
testset = torchvision.datasets.CIFAR10(root='./data', train = False, download=True, transform=transform)
testset_1 = torch.utils.data.Subset(testset, train)
testloader_1 = torch.utils.data.DataLoader(testset_1, shuffle=True, num_workers=4,batch_size = 20)
print(len(trainset_1))
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

AttributeError: ignored

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']
# helper function to un-normalize and display an image
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    plt.imshow(np.transpose(img, (1, 2, 0)))  # convert from Tensor image
# obtain one batch of training images
dataiter = iter(trainloader_1)
images, labels = next(dataiter)
images = images.numpy() # convert images to numpy for display
# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
# display 20 images
for idx in np.arange(20):
  ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])
  imshow(images[idx])
  ax.set_title(classes[labels[idx]])

CNN

In [ ]:
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):


    def __init__(self):

        super(CNN, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),
            nn.GroupNorm(4,32),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.GroupNorm(4,128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),
            nn.GroupNorm(4,256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )


        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        """Perform forward."""

        # conv layers
        x = self.conv_layer(x)

        # flatten
        x = x.view(x.size(0), -1)

        # fc layer
        x = self.fc_layer(x)

        return x
model = CNN()
print(model)
#print(model.parameters())
criterion = nn.CrossEntropyLoss()
# specify optimizer
optimizer = optim.SGD(model.parameters(), lr=.001)

CNN(
  (conv_layer): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Dropout2d(p=0.05, inplace=False)
    (13): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stat

model 1:

In [ ]:
train_losslist = []
# number of epochs to train the model
n_epochs = [*range(30)] # you may increase this number to train a final model

valid_loss_min = np.Inf # track change in validation loss

for epoch in n_epochs:

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0

    ###################
    # train the model #
    ###################
    model.train()
    for data, target in trainloader_1:
        # move tensors to GPU if CUDA is available
       # if train_on_gpu:
           # data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)

    ######################
    # validate the model #
    ######################
    model.eval()
    for data, target in validloader:
        # move tensors to GPU if CUDA is available
        #if train_on_gpu:
            #data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss
        valid_loss += loss.item()*data.size(0)

    # calculate average losses
    train_loss = train_loss/len(trainloader_1.dataset)
    valid_loss = valid_loss/len(validloader.dataset)

    train_losslist.append(train_loss)

    # print training/validation statistics
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))

    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

Epoch: 0 	Training Loss: 2.301538 	Validation Loss: 2.299052
Validation loss decreased (inf --> 2.299052).  Saving model ...
Epoch: 1 	Training Loss: 2.296779 	Validation Loss: 2.294651
Validation loss decreased (2.299052 --> 2.294651).  Saving model ...
Epoch: 2 	Training Loss: 2.292773 	Validation Loss: 2.290282
Validation loss decreased (2.294651 --> 2.290282).  Saving model ...
Epoch: 3 	Training Loss: 2.287542 	Validation Loss: 2.285661
Validation loss decreased (2.290282 --> 2.285661).  Saving model ...
Epoch: 4 	Training Loss: 2.282698 	Validation Loss: 2.280748
Validation loss decreased (2.285661 --> 2.280748).  Saving model ...
Epoch: 5 	Training Loss: 2.276447 	Validation Loss: 2.274786
Validation loss decreased (2.280748 --> 2.274786).  Saving model ...
Epoch: 6 	Training Loss: 2.269247 	Validation Loss: 2.267651
Validation loss decreased (2.274786 --> 2.267651).  Saving model ...
Epoch: 7 	Training Loss: 2.261552 	Validation Loss: 2.259769
Validation loss decreased (2.26765

In [ ]:
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# iterate over test data
for data, target in testloader_1:
    # move tensors to GPU if CUDA is available
    #data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(20):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(testloader_1.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 1.713490

Test Accuracy of airplane: 57% (101/176)
Test Accuracy of automobile: 51% (94/184)
Test Accuracy of  bird: 23% (41/174)
Test Accuracy of   cat:  1% ( 2/180)
Test Accuracy of  deer: 36% (66/183)
Test Accuracy of   dog: 12% (21/167)
Test Accuracy of  frog: 48% (93/190)
Test Accuracy of horse: 44% (78/175)
Test Accuracy of  ship: 47% (90/189)
Test Accuracy of truck: 43% (79/182)

Test Accuracy (Overall): 36% (665/1800)


model 2

comparison and saving

In [ ]:
from numpy.linalg import norm
params1 = []
norms1 = []
for param in m.parameters():
   # print(param)
    temp = param.detach().numpy()
    norms1.append(norm(temp))
    print(np.shape(temp))
    params1.append(temp)
print(np.shape(params1))
print(norm(norms1))


(32, 3, 3, 3)
(32,)
(32,)
(32,)
(64, 32, 3, 3)
(64,)
(128, 64, 3, 3)
(128,)
(128,)
(128,)
(128, 128, 3, 3)
(128,)
(256, 128, 3, 3)
(256,)
(256,)
(256,)
(256, 256, 3, 3)
(256,)
(1024, 4096)
(1024,)
(512, 1024)
(512,)
(10, 512)
(10,)
(24,)


given an arbitrary data point removed, how much does output change

In [ ]:
outfile1 = TemporaryFile()
from numpy.linalg import norm
params2 = []
for param in model.parameters():
   # print(param)
    temp = param.detach().numpy()
    print(np.shape(temp))
    params2.append(temp)
print(np.shape(params2))


(32, 3, 3, 3)
(32,)
(32,)
(32,)
(64, 32, 3, 3)
(64,)
(128, 64, 3, 3)
(128,)
(128,)
(128,)
(128, 128, 3, 3)
(128,)
(256, 128, 3, 3)
(256,)
(256,)
(256,)
(256, 256, 3, 3)
(256,)
(1024, 4096)
(1024,)
(512, 1024)
(512,)
(10, 512)
(10,)
(24,)


In [ ]:
from numpy import linalg as LA
results = []
for x in range(24):
  temp = params1[x]-params2[x]
  results.append(LA.norm(temp))

print(LA.norm(results))

40.390415
